<a href="https://colab.research.google.com/github/luckycontrol/DeepLearning_tensorflow/blob/main/6_%EC%97%AD%EC%A0%84%ED%8C%8C_%EA%B5%AC%ED%98%84%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 곱셈계층
class MulLayer:

  # 초기화 : 객체에서 사용할 변수를 미리 준비
  def __init__(self):
    self.x = None
    self.y = None

  # x, y를 곱해서 리턴.
  # 계산되는 값들을 저장하고 있어야 한다.
  def forward(self, x, y):
    self.x = x
    self.y = y

    out = x * y
    return out

  # dout : 뒤에서 넘어온 미분값.
  # dx = dout * y
  # dy = dout * x
  def backward(self, dout):
    dx = dout * self.y
    dy = dout * self.x

    return dx, dy

In [ ]:
apple = 100 # 사과 갯수 1개의 가격
apple_cnt = 2 # 사과의 갯수
tax = 1.1 # 소비세

# 계층은 2개
# (apple * apple_cnt) * tax
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

apple_price = mul_apple_layer.forward(apple, apple_cnt)
price = mul_tax_layer.forward(apple_price, tax)

print("최종 사과 가격: {:.0f}".format(price))

최종 사과 가격: 220


In [ ]:
# 최종가격에 대한 미분
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_cnt = mul_apple_layer.backward(dapple_price)

print("사과 가격의 미분값: {}".format(dapple))
print("사과 갯수의 미분값: {}".format(dapple_cnt))
print("소비세의 미분값: {}".format(dtax))

사과 가격의 미분값: 2.2
사과 갯수의 미분값: 110.00000000000001
소비세의 미분값: 200


In [ ]:
# 덧셈계층 구현하기

# forward : x + y
# backward : 뒤에서 보내온 미분값 * 1 ( 모양 맞추려고.. )
# 비고 : forward 에서의 입력값을 가지고 있지 않아도 된다. 왜? 역전파 할 때 미분값만 필요하니까!

class AddLayer:
  def __init__(self):
    pass

  def forward(self, x, y):
    out = x + y
    return out

  def backward(self, dout):
    dx = dout * 1
    dy = dout * 1

    return dx, dy

In [ ]:
apple_cnt = 2
apple_price = 100

orange_price = 150
orange_cnt = 3

tax = 1.1

In [ ]:
apple_cnt_layer = MulLayer()
orange_cnt_layer = MulLayer()
price_layer = AddLayer()
total_price_layer = MulLayer()

In [ ]:
apple_price = apple_cnt_layer.forward(apple_cnt, apple_price)
orange_price = orange_cnt_layer.forward(orange_cnt, orange_price)
price = price_layer.forward(apple_price, orange_price)
total_price = total_price_layer.forward(price, tax)

print("총 가격: {:.0f}".format(price))

총 가격: 650


In [ ]:
print("총 가격: {:.0f}".format(total_price))

총 가격: 715


In [ ]:
dprice = 1

dtotal_price, dtax = total_price_layer.backward(dprice)
dapple_total_price, dorange_total_price = price_layer.backward(dtotal_price)

dapple_cnt, dapple_price = apple_cnt_layer.backward(dapple_total_price)
dorange_cnt, dorange_price = orange_cnt_layer.backward(dorange_total_price)

print("사과 갯수 미분: {}".format(dapple_cnt))
print("사과 가격 미분: {}".format(dapple_price))

print("오렌지 갯수 미분: {}".format(dorange_cnt))
print("오렌지 가격 미분: {}".format(dorange_price))

print("소비세 미분: {}".format(dtax))

사과 갯수 미분: 110.00000000000001
사과 가격 미분: 2.2
오렌지 갯수 미분: 165.0
오렌지 가격 미분: 3.3000000000000003
소비세 미분: 650


# 신경망 레이어 만들기

- ReLU
- Sigmoid
- Affine 레이어 ( 기하학 레이어 - Fully Connected, Dense )
- SoftMax + Loss 레이어

In [ ]:
# 순정파 때 학습이 안된 데이터는, 역전파 때도 신경쓰지 않겠다.
# 알아내고자 하는 특징만 강조할 수 있기에 이미지분류에 많이 사용된다.
class ReLU:

  # 가지고 있어야 할 정보 : 어떤 위치의 x가 0보다 작았나?? ( masking )
  def __init__(self):
    self.mask = None

  def forward(self, x):
    self.mask = (x <= 0) # 배열 x에서 어떤 값이 0 또는 음수인지 확인 - 음수인 부분만 True

    out = x.copy() # 원본 데이터는 손상되면 안돼요.
    out[self.mask] = 0 # 0 또는 음수인 인덱스에 0을 넣는다.

    return out

  # 순전파 때 0 또는 음수였던 부분을 0으로 만들었다.
  # 0 또는 음수였던 인덱스를 기억해놨다가 미분값을 전달 받았을 때 해당 인덱스를 0으로 만든다.
  def backward(self, dout):
    dout[self.mask] = 0
    dx = dout
    return dx

In [ ]:
import numpy as np

x = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)

[[ 1.  -0.5]
 [-2.   3. ]]


In [ ]:
relu = ReLU()

relu.forward(x)

array([[1., 0.],
       [0., 3.]])

In [ ]:
# 마스크 확인
relu.mask

array([[False,  True],
       [ True, False]])

In [ ]:
dx = np.array([[-0.1, 4.0], [1.3, -1.1]])

relu.backward(dx)

array([[-0.1,  0. ],
       [ 0. , -1.1]])

In [ ]:
class Sigmoid:
  def __init__(self):
    self.out = None # sigmoid 순전파에서 계산된 값 ( y값 )

  def forward(self, x):
    out = 1 / (1 + np.exp(-x))
    self.out = out

    return out

  # 시그모이드 함수를 미분하면 : sigmoid * (1 - sigmoid)
  def backward(self, dout):
    dx = dout * (1.0 - self.out) * self.out
    return dx

In [ ]:
x = np.array([0.991, 0.34, 0.56])
sigmoid = Sigmoid()

print(sigmoid.forward(x))

dout = 1
print(sigmoid.backward(dout))

[0.7292854  0.58419052 0.63645254]
[0.19742821 0.24291196 0.2313807 ]


In [ ]:
# Affine 계층의 순전파
# Affine의 핵심은 원본 모양을 가지고 있어야 한다는 것! ( 기하학 변환 )
x = np.random.rand(2)
W = np.random.rand(2, 3)
H = np.random.rand(3)

y = np.dot(x, W) + H
print(y)

[0.15853231 0.3319097 ]
[0.57144708 1.28722861 0.86853127]


In [ ]:
x.shape, W.shape, H.shape

((2,), (2, 3), (3,))

In [ ]:
x_dot_w = np.array([[0, 0, 0], [10, 10, 10]]) # X와 W의 내적 결과
B = np.array([1, 2, 3])

print(x_dot_w + B) # B에서 브로드캐스팅이 발생

[[ 1  2  3]
 [11 12 13]]


In [ ]:
dy = np.array([[1, 2, 3], 
               [4, 5, 6]]) # 각 배치에 대한 미분값

dB = np.sum(dy, axis=0)
print(dB)

[5 7 9]


In [ ]:
class Affine:
  def __init__(self, w, b):
    self.w = w
    self.b = b

    self.x = None
    self.original_x_shape = None

    # w, b의 최적화를 위해서 각 매개변수의 미분값을 갖고 있어야 함. ( 가중치와 편향 update )
    self.dW = None
    self.db = None

  def forward(self, x):
    # 배치마다의 데이터를 1열로 만듦.
    # ex) (2, 2, 2) -> (2, 4)
    self.original_x_shape = x.shape
    x = x.reshape(x.shape[0], -1) # 첫 번째 : 배치 크기

    self.x = x

    out = np.dot(self.x, self.w) + self.b
    return out

  def backward(self, dout):
    dx = np.dot(dout, self.w.T)
    self.dW = np.dot(self.w.T, dout)
    self.db = np.sum(dout, axis=0)

    dx = dx.reshape(*self.original_x_shape)
    return dx

In [ ]:
def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T 

    x = x - np.max(x) # 오버플로 대책
    return np.exp(x) / np.sum(np.exp(x))


def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    # 훈련 데이터가 원-핫 벡터라면 정답 레이블의 인덱스로 반환
    if t.size == y.size:
        t = t.argmax(axis=1)
             
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

# 출력층을 위한 활성화 계층
class SoftmaxWithLoss:
  def __init__(self):
    self.loss = None
    self.y = None
    self.t = None

  def forward(self, x, t):
    self.t = t
    self.y = softmax(x)
    self.loss = cross_entropy_error(self.y, self.t)

    return self.loss

  def backward(self, dout=1):
    batch_size = self.t.shape[0] # x는 계속 바뀌니까..

    if self.t.size == self.y.size:
      dx = (self.y - self.t) / batch_size

    # t가 OneHotEncoding이 안되어 있는 경우
    else:
      dx = self.y.copy() # 예측값 복사
      dx[np.arange(batch_size), self.t] -= 1
      dx = dx / batch_size

    return dx


In [ ]:
%cd common
!unzip common.zip

/content/common
Archive:  common.zip
  inflating: functions.py            
  inflating: gradient.py             
  inflating: layers.py               
  inflating: multi_layer_net.py      
  inflating: multi_layer_net_extend.py  
  inflating: optimizer.py            
  inflating: trainer.py              
  inflating: util.py                 
 extracting: __init__.py             


In [ ]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict


class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
        self.params['b2'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict() # 추가되는 데이터의 순서가 유지가 된다.
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
        
    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
    def gradient(self, x, t):
        # 순전파
        self.loss(x, t)

        # 역전파
        dout = 1 # 마지막 계층의 미분값 설정
        dout = self.lastLayer.backward(dout) # 마지막 계층에서의 미분값 전달 받기 (SoftMaxWithLoss에서 받음)
        
        layers = list(self.layers.values()) # 저장된 레이어를 불러와서 ( 여기서는 순차적인 레이어가 저장 되어 있음 )
        layers.reverse() # 뒤집음(뒤에서 부터 전달해야 하기 때문에 )
        
        # 뒤에서 부터 역전파
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads